# Bibliotecas

# função objetivo

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

# === SUAS FUNÇÕES REAIS ===
from my_example import *

# === PYMOO ===
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize

# -*- coding: utf-8 -*-
"""
Otimização de fundações com pymoo (GA)
- Lê 'teste_reduzido.xlsx'
- Importa funções reais de 'my_example.py'
- Define obj(x) SEM estado global
- Monta g_0..g_15 + penalização agregada df['g']
- Roda GA, e se não houver solução factível, devolve a melhor por CV (menor violação)
"""


# ==========================
# 0) DADOS BASE IMUTÁVEIS
# ==========================
N_COMB = 3  # c1, c2, c3
PENAL = 1e6

df_base = tensao_adm_solo(pd.read_excel("teste_sem_sobreposicao.xlsx")).reset_index(drop=True)
N_FUNDACOES = len(df_base)

# Checagens mínimas
for k in range(1, N_COMB + 1):
    for name in (f"Fz-c{k}", f"Mx-c{k}", f"My-c{k}"):
        if name not in df_base.columns:
            raise RuntimeError(f"Coluna obrigatória ausente no Excel: '{name}'")
need_cols = ["xg (m)", "yg (m)", "ap (m)", "bp (m)", "sigma_adm (kPa)"]
miss = [c for c in need_cols if c not in df_base.columns]
if miss:
    raise RuntimeError("Colunas obrigatórias ausentes no Excel: " + ", ".join(miss))

# ==============================
# 1) FUNÇÃO-OBJETIVO (SEM ESTADO)
# ==============================
def obj(x: np.ndarray):
    """
    Retorna:
        F (float): soma dos volumes
        G (np.ndarray [N_FUNDACOES,]): penalização agregada por fundação (>=0 quando viola)
    """
    df = df_base.copy(deep=True)

    hx = np.asarray(x[0:N_FUNDACOES], dtype=float)
    hy = np.asarray(x[N_FUNDACOES:2 * N_FUNDACOES], dtype=float)
    hz = np.asarray(x[2 * N_FUNDACOES:3 * N_FUNDACOES], dtype=float)

    df["h_x (m)"] = hx
    df["h_y (m)"] = hy
    df["h_z (m)"] = hz

    # Volume
    df["volume (m3)"] = df["h_x (m)"] * df["h_y (m)"] * df["h_z (m)"]

    # Tensões críticas
    max_tensoes, min_tensoes = [], []
    rotulos_comb = [f"c{i}" for i in range(1, N_COMB + 1)]
    idx_crit_max, idx_crit_min = [], []
    fz_max = []; mx_max = []; my_max = []
    fz_min = []; mx_min = []; my_min = []

    for _, row in df.iterrows():
        tmax_aux, tmin_aux = [], []
        for i in range(1, N_COMB + 1):
            smax, smin = calcular_sigma_max(
                row[f"Fz-c{i}"], row[f"Mx-c{i}"], row[f"My-c{i}"],
                row["h_x (m)"], row["h_y (m)"]
            )
            tmax_aux.append(smax); tmin_aux.append(smin)

        mxv = max(tmax_aux)
        mnv = min(tmin_aux)
        max_tensoes.append(mxv)
        min_tensoes.append(mnv)
        imax = int(np.argmax(tmax_aux))
        imin = int(np.argmin(tmin_aux))
        idx_crit_max.append(rotulos_comb[imax]); idx_crit_min.append(rotulos_comb[imin])
        comb_max = rotulos_comb[imax]; comb_min = rotulos_comb[imin]
        fz_max.append(row[f"Fz-{comb_max}"]); mx_max.append(row[f"Mx-{comb_max}"]); my_max.append(row[f"My-{comb_max}"])
        fz_min.append(row[f"Fz-{comb_min}"]); mx_min.append(row[f"Mx-{comb_min}"]); my_min.append(row[f"My-{comb_min}"])

    df["max_tensao (kPa)"] = np.asarray(max_tensoes, dtype=float)
    df["min_tensao (kPa)"] = np.asarray(min_tensoes, dtype=float)

    df["comb_max_tensao"] = idx_crit_max
    df["Fz_max (kN)"] = np.asarray(fz_max, dtype=float)
    df["Mx_max (kN.m)"] = np.asarray(mx_max, dtype=float)
    df["My_max (kN.m)"] = np.asarray(my_max, dtype=float)
    df["comb_min_tensao"] = idx_crit_min
    df["Fz_min (kN)"] = np.asarray(fz_min, dtype=float)
    df["Mx_min (kN.m)"] = np.asarray(mx_min, dtype=float)
    df["My_min (kN.m)"] = np.asarray(my_min, dtype=float)

    #Restrições geom. (balanço)
    g0_geo_ps, g1_geo_ps, g2_geo_ps, g3_geo_ps = [], [], [], []
    cap_list, cbp_list = [], []
    for _, row in df.iterrows():
        cap, cbp, g0, g1, g2, g3 = restricao_geometrica_balanco_pilar_sapata(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"], h_z=row["h_z (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"]
        )
        cap_list.append(cap); cbp_list.append(cbp)
        g0_geo_ps.append(g0); g1_geo_ps.append(g1)
        g2_geo_ps.append(g2); g3_geo_ps.append(g3)

    df["cap (m)"] = np.asarray(cap_list, dtype=float)
    df["cbp (m)"] = np.asarray(cbp_list, dtype=float)

    df["g_2"] = np.asarray(g0_geo_ps, dtype=float)
    df["g_3"] = np.asarray(g1_geo_ps, dtype=float)
    df["g_4"] = np.asarray(g2_geo_ps, dtype=float)
    df["g_5"] = np.asarray(g3_geo_ps, dtype=float)

    # Restrições geom. pilar-sapata
    g0_geo_p, g1_geo_p = [], []
    for _, row in df.iterrows():
        g0, g1 = restricao_geometrica_pilar_sapata(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"]
        )
        g0_geo_p.append(g0); g1_geo_p.append(g1)
    df["g_6"] = np.asarray(g0_geo_p, dtype=float)
    df["g_7"] = np.asarray(g1_geo_p, dtype=float)

    # Punção – usa comb. crítica de máximo
    ro = 0.01; cob = 0.025; fck = 25000.0; fcd = fck / 1.4

    gp0_list= []; gp1_list= []; gp2_list= []; gp3_list= []
    gp4_list= []; gp5_list= []; gp6_list= []
    # (coeficientes/auxiliares guardados apenas por transparência)
    for _, row in df.iterrows():
        ke, kx, ky, wpx, wpy, u, talsd1, talrd1, talsd2, talrd2, g0, g1, g2, g3, g4, g5, g6 = restricao_puncao(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"], h_z=row["h_z (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"],
            f_z=row["Fz_max (kN)"], m_x=row["Mx_max (kN.m)"], m_y=row["My_max (kN.m)"],
            ro=ro, cob=cob, fck=fck, fcd=fcd
        )
        gp0_list.append(g0); gp1_list.append(g1); gp2_list.append(g2)
        gp3_list.append(g3); gp4_list.append(g4); gp5_list.append(g5); gp6_list.append(g6)

    df["g_8"]  = np.asarray(gp0_list, dtype=float)
    df["g_9"]  = np.asarray(gp1_list, dtype=float)
    df["g_10"] = np.asarray(gp2_list, dtype=float)
    df["g_11"] = np.asarray(gp3_list, dtype=float)
    df["g_12"] = np.asarray(gp4_list, dtype=float)
    df["g_13"] = np.asarray(gp5_list, dtype=float)
    df["g_14"] = np.asarray(gp6_list, dtype=float)

    # Interseção
    xa = df["xg (m)"] + df["h_x (m)"] / 2.0
    ya = df["yg (m)"] + df["h_y (m)"] / 2.0
    xb = df["xg (m)"] - df["h_x (m)"] / 2.0
    yb = df["yg (m)"] - df["h_y (m)"] / 2.0
    xc = df["xg (m)"] - df["h_x (m)"] / 2.0
    yc = df["yg (m)"] + df["h_y (m)"] / 2.0
    xd = df["xg (m)"] + df["h_x (m)"] / 2.0
    yd = df["yg (m)"] - df["h_y (m)"] / 2.0

    polys = []
    for i in range(len(df)):
        pts = [(xa.iloc[i], ya.iloc[i]), (xb.iloc[i], yb.iloc[i]),
               (xc.iloc[i], yc.iloc[i]), (xd.iloc[i], yd.iloc[i]), (xa.iloc[i], ya.iloc[i])]
        polys.append(Polygon(pts))

    inter_count = np.zeros(len(df), dtype=float)
    for i in range(len(df)):
        for j in range(len(df)):
            if i != j and polys[i].intersects(polys[j]):
                inter_count[i] += 1.0
    df["g_15"] = inter_count

    # Tensões -> g_0 e g_1 (sua fórmula)
    df["g_0"] = np.where(
        df["min_tensao (kPa)"] < 0.0,
        np.abs(df["min_tensao (kPa)"] / df["sigma_adm (kPa)"]),
        1.15 * df["min_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
    ).astype(float)
    df["g_1"] = np.where(
        df["max_tensao (kPa)"] < 0.0,
        np.abs(df["max_tensao (kPa)"] / df["sigma_adm (kPa)"]),
        1.15 * df["max_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
    ).astype(float)

    # Garantir todas g_0..g_15
    req_cols = [f"g_{k}" for k in range(16)]
    missing = [c for c in req_cols if c not in df.columns]
    if missing:
        raise RuntimeError("Colunas de restrição não geradas: " + ", ".join(missing))

    # Penalização agregada por fundação
    g_mat = df[req_cols].to_numpy(dtype=float)
    g_pos = np.where(g_mat > 0.0, g_mat, 0.0)
    df["g"] = PENAL * np.sum(g_pos, axis=1)

    F = float(np.sum(df["volume (m3)"].to_numpy(dtype=float)))
    G = df["g"].to_numpy(dtype=float).reshape(-1)

    if not np.isfinite(F) or np.any(~np.isfinite(G)):
        raise RuntimeError("F ou G contém NaN/Inf. Verifique cálculos intermediários.")

    return F, G


# ==============================
# 2) PROBLEMA PYMOO
# ==============================
class MyProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=N_FUNDACOES * 3,
            n_obj=1,
            n_constr=N_FUNDACOES,
            xl=np.array([1] * N_FUNDACOES * 3, dtype=float),
            xu=np.array([5.0] * N_FUNDACOES * 3, dtype=float),
        )

    def _evaluate(self, x, out, *args, **kwargs):
        f, g = obj(x)
        out["F"] = float(f)
        out["G"] = np.asarray(g, dtype=float).reshape(-1)


# ==============================
# 3) SEEDING AMIGÁVEL + GA
# ==============================
def build_seed():
    # Gera um chute inicial cumprindo (em geral) geometria mínima
    hx0 = np.maximum(0.6, df_base["ap (m)"].to_numpy(dtype=float) + 0.20)
    hy0 = np.maximum(0.6, df_base["bp (m)"].to_numpy(dtype=float) + 0.20)
    hz0 = np.full(N_FUNDACOES, 0.60, dtype=float)
    x0 = np.hstack([hx0, hy0, hz0])
    x0 = np.clip(x0, 0.6, 3.0)
    return x0

def inject_seed(pop_array, seed):
    # Substitui a primeira solução pela semente
    pop_array[0, :] = seed
    return pop_array

if __name__ == "__main__":
    problem = MyProblem()
    algorithm = GA(
        pop_size=40,                 # ligeiro aumento da população
        n_offsprings=20,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PM(eta=20),
        eliminate_duplicates=True,
    )

    # Executa
    res = minimize(
        problem,
        algorithm,
        ("n_gen", 40),               # mais gerações p/ chance de viabilidade
        seed=1,
        verbose=False,
        save_history=True,
    )

    # ==============================
    # 4) EXTRAÇÃO ROBUSTA DO “MELHOR”
    # ==============================
    def get_best_solution(res):
        # Se houve solução factível:
        if res.F is not None and res.X is not None:
            return res.X, res.F, res.G, True

        # Caso contrário, pega a de menor violação (CV) da última população
        pop = getattr(res, "pop", None)
        if pop is None and hasattr(res, "history") and len(res.history) > 0:
            pop = res.history[-1].pop
        if pop is None:
            return None, None, None, False

        CV = pop.get("CV").ravel()
        I = int(np.argmin(CV))
        Xb = pop[I].get("X")
        Fb = pop[I].get("F")
        Gb = pop[I].get("G")
        return Xb, Fb, Gb, False

    X, F, G, feasible = get_best_solution(res)

    print("\n================ RESULTADOS ================")
    print("Encontrou solução factível?", feasible)
    if X is not None:
        print("Melhor F (volume total):", F)
        if G is not None:
            print("Max G:", np.max(G))
            print("Todas restrições satisfeitas? (G <= 0):", np.all(G <= 0))
        print("Vetor X (h_x ... h_y ... h_z ...):\n", X)
        try:
            np.savetxt("melhor_X.csv", np.atleast_2d(X), delimiter=";", fmt="%.6f")
            print("Arquivo 'melhor_X.csv' salvo.")
        except Exception as e:
            print("Não foi possível salvar melhor_X.csv:", e)
    else:
        print("Não foi possível recuperar solução (nem por CV). Revise restrições/dados.")



================ RESULTADOS ================
Encontrou solução factível? False
Melhor F (volume total): [276.65087446]
Max G: 3716821.1565089924
Todas restrições satisfeitas? (G <= 0): False
Vetor X (h_x ... h_y ... h_z ...):
 [2.60697236 2.14813987 2.28771389 2.73924007 3.34684704 1.48618488
 1.33115193 1.6977395  2.51289953 3.04544645 4.03697424 4.20735809
 1.90167129 1.83212745 1.09466451 3.32618956 3.42062614 4.21281141
 3.98261224 4.01045092 3.15884337 4.60171191 4.57589401 2.87058027
 2.13267608 3.29181798 1.7607466  2.58124303 3.2665899  4.14992422
 4.08202045 4.36387343 2.05649392 2.62126726 2.02209299 3.03478086
 1.43394299 1.62319464 3.37024147 2.56470537 1.05938247 1.87759457
 1.80790906 1.0661382  2.01354234 1.07672652 2.05106724 1.83260509
 1.08534872 1.84350941 2.50591593 2.26398146 1.69930082 1.49024542]
Arquivo 'melhor_X.csv' salvo.


# Verificando restrição baçamço

In [37]:
import pandas as pd
from my_example import restricao_geometrica_balanco_pilar_sapata

# 1) Ler o arquivo Excel
df = pd.read_excel("teste_reduzido.xlsx")

h_x = 2, 2, 2, 2, 0.1
h_y = 2, 3, 3, 3, 0.1
h_z = .49
df['h_x (m)'] = h_x
df['h_y (m)'] = h_y
df['h_z (m)'] = h_z

# 2) Garantir que as colunas tenham nomes corretos
# (ajuste se no seu Excel estiverem diferentes)
colunas_esperadas = ['h_x (m)', 'h_y (m)', 'h_z (m)', 'ap (m)', 'bp (m)']
for c in colunas_esperadas:
    if c not in df.columns:
        raise KeyError(f"A coluna {c} não foi encontrada no arquivo Excel.")

# 3) Aplicar a função linha a linha e guardar resultados
resultados = df.apply(
    lambda row: restricao_geometrica_balanco_pilar_sapata(
        row['h_x (m)'],
        row['h_y (m)'],
        row['h_z (m)'],
        row['ap (m)'],
        row['bp (m)']
    ),
    axis=1
)

# 4) Separar o resultado em novas colunas
df[['cap_value', 'cbp_value', 'g_0', 'g_1', 'g_2', 'g_3']] = pd.DataFrame(resultados.tolist(), index=df.index)

# 5) Exibir os primeiros resultados
print(df[['cap_value', 'cbp_value', 'g_0', 'g_1', 'g_2', 'g_3']].head())

# 6) (Opcional) salvar em Excel
# df.to_excel("resultado_balanco.xlsx", index=False)
# print("Arquivo 'resultado_balanco.xlsx' salvo com sucesso.")


   cap_value  cbp_value       g_0       g_1       g_2       g_3
0      0.875       0.40 -0.107143 -0.591837 -0.720000 -0.387500
1      0.850       0.75 -0.132653 -0.234694 -0.711765 -0.673333
2      0.825       0.34 -0.158163 -0.653061 -0.703030 -0.279412
3      0.850       0.75 -0.132653 -0.234694 -0.711765 -0.673333
4     -0.075      -0.55 -1.076531 -1.561224 -4.266667 -1.445455


# Verificando restrição sobreposição

In [35]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon  # se preferir, dá pra usar shapely.geometry.box

ARQ_IN = "teste_reduzido.xlsx"
ARQ_OUT = "resultado_intersecoes.xlsx"

# 1) Ler o arquivo
df = pd.read_excel(ARQ_IN)
h_x = 2, 1, 4, 5, 0.1
h_y = 2, 4, 5, 4, 0.1
h_z = .49
df['h_x (m)'] = h_x
df['h_y (m)'] = h_y
df['h_z (m)'] = h_z

# 2) Conferir colunas necessárias
cols = ['xg (m)', 'yg (m)', 'h_x (m)', 'h_y (m)']
faltando = [c for c in cols if c not in df.columns]
if faltando:
    raise KeyError(f"Colunas ausentes no Excel: {faltando}")

# 3) Calcular vértices (conforme seu código)
xa = df["xg (m)"] + df["h_x (m)"] / 2.0
ya = df["yg (m)"] + df["h_y (m)"] / 2.0
xb = df["xg (m)"] - df["h_x (m)"] / 2.0
yb = df["yg (m)"] - df["h_y (m)"] / 2.0
xc = df["xg (m)"] - df["h_x (m)"] / 2.0
yc = df["yg (m)"] + df["h_y (m)"] / 2.0
xd = df["xg (m)"] + df["h_x (m)"] / 2.0
yd = df["yg (m)"] - df["h_y (m)"] / 2.0

# 4) Montar os polígonos (retângulos) linha a linha
polys = []
for i in range(len(df)):
    pts = [
        (xa.iloc[i], ya.iloc[i]),
        (xb.iloc[i], yb.iloc[i]),
        (xc.iloc[i], yc.iloc[i]),
        (xd.iloc[i], yd.iloc[i]),
        (xa.iloc[i], ya.iloc[i]),
    ]
    polys.append(Polygon(pts))

# 5) Contar interseções (O(n^2) simples e fiel ao seu código)
n = len(polys)
inter_count = np.zeros(n, dtype=float)

for i in range(n):
    for j in range(n):
        if i != j and polys[i].intersects(polys[j]):
            inter_count[i] += 1.0

# 6) Gravar no DataFrame
df["g_15"] = inter_count

# 7) Visualizar e salvar
print(df[["xg (m)", "yg (m)", "h_x (m)", "h_y (m)", "g_15"]].head())
# df.to_excel(ARQ_OUT, index=False)
# print(f"Arquivo '{ARQ_OUT}' salvo com sucesso.")



   xg (m)  yg (m)  h_x (m)  h_y (m)  g_15
0   2.900  25.255      2.0      2.0   0.0
1   7.985  25.105      1.0      4.0   0.0
2  13.875  25.515      4.0      5.0   0.0
3  24.850  25.105      5.0      4.0   0.0
4  24.850  25.255      0.1      0.1   0.0


# Verificando restrição de tensão

In [ ]:
import pandas as pd
import numpy as np

ARQ_IN = "teste_reduzido.xlsx"
ARQ_OUT = "resultado_tensoes.xlsx"

# 1) Ler a planilha
df = pd.read_excel(ARQ_IN)
df = tensao_adm_solo(df)

h_x = 2, 1, 4, 5, 0.1
h_y = 2, 4, 5, 4, 0.1
h_z = .49
df['h_x (m)'] = h_x
df['h_y (m)'] = h_y
df['h_z (m)'] = h_z



df['min_tensao (kPa)'], df['max_tensao (kPa)'] = calcular_sigma_max(f_z, m_x, m_y, h_x, h_y)

# df['min_tensao (kPa)'] = 315, 520, 408, 530, 330
# df['sigma_adm (kPa)'] = 200, 240, 600, 700, 900
# df['max_tensao (kPa)'] = 710, 1305, 3000, 1255, 680

# 2) Conferir se as colunas necessárias existem
cols = ["min_tensao (kPa)", "max_tensao (kPa)", "sigma_adm (kPa)"]
faltando = [c for c in cols if c not in df.columns]
if faltando:
    raise KeyError(f"Colunas ausentes no Excel: {faltando}")

# 3) Aplicar as fórmulas fornecidas
df["g_0"] = np.where(
    df["min_tensao (kPa)"] < 0.0,
    np.abs(df["min_tensao (kPa)"] / df["sigma_adm (kPa)"]),
    1.15 * df["min_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
).astype(float)

df["g_1"] = np.where(
    df["max_tensao (kPa)"] < 0.0,
    np.abs(df["max_tensao (kPa)"] / df["sigma_adm (kPa)"]),
    1.15 * df["max_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
).astype(float)

# 4) Exibir os primeiros resultados
print(df[["min_tensao (kPa)", "max_tensao (kPa)", "sigma_adm (kPa)", "g_0", "g_1"]].head())

# 5) Salvar o resultado em Excel
df.to_excel(ARQ_OUT, index=False)
print(f"Arquivo '{ARQ_OUT}' salvo com sucesso.")


   min_tensao (kPa)  max_tensao (kPa)  sigma_adm (kPa)       g_0       g_1
0               315               710              200  0.811250  3.082500
1               520              1305              240  1.491667  5.253125
2               408              3000              600 -0.218000  4.750000
3               530              1255              700 -0.129286  1.061786
4               330               680              900 -0.578333 -0.131111
Arquivo 'resultado_tensoes.xlsx' salvo com sucesso.


# verificando pução

In [42]:
import pandas as pd
import numpy as np
from my_example import tensao_adm_solo, calcular_sigma_max  # <- usa suas defs

ARQ_IN  = "teste_reduzido.xlsx"
ARQ_OUT = "resultado_tensoes.xlsx"

# 1) Ler a planilha e calcular sigma_adm
df = pd.read_excel(ARQ_IN)
df = tensao_adm_solo(df)

# 2) Dimensões: use LISTAS alinhadas ao número de linhas (não tuplas soltas)
hx_list = [2, 1, 4, 5, 0.1]
hy_list = [2, 4, 5, 4, 0.1]
hz_val  = 0.49  # escalar serve para todas as linhas

# Ajusta o tamanho das listas para bater com o df:
n = len(df)
if len(hx_list) != n or len(hy_list) != n:
    # Estratégia simples: repetir até ter n elementos e então truncar
    if len(hx_list) < n:
        k = (n + len(hx_list) - 1) // len(hx_list)
        hx_list = (hx_list * k)[:n]
    else:
        hx_list = hx_list[:n]
    if len(hy_list) < n:
        k = (n + len(hy_list) - 1) // len(hy_list)
        hy_list = (hy_list * k)[:n]
    else:
        hy_list = hy_list[:n]

df['h_x (m)'] = hx_list
df['h_y (m)'] = hy_list
df['h_z (m)'] = hz_val

# 3) Esforços: calcular tensões com calcular_sigma_max SE houver Fz/Mx/My
tem_forcas = all(c in df.columns for c in ['Fz', 'Mx', 'My'])
if tem_forcas:
    # ATENÇÃO: calcular_sigma_max(f_z, m_x, m_y, h_x, h_y) -> (sigma_max, sigma_min)
    max_min = df.apply(
        lambda r: calcular_sigma_max(
            float(r['Fz']),
            float(r['Mx']),
            float(r['My']),
            float(r['h_x (m)']),
            float(r['h_y (m)'])
        ),
        axis=1
    )
    # 'max_min' é uma Série de tuplas; transformamos em duas colunas
    df[['max_tensao (kPa)', 'min_tensao (kPa)']] = pd.DataFrame(max_min.tolist(), index=df.index)
else:
    # Se você AINDA não tem Fz/Mx/My na planilha,
    # defina manualmente as tensões (exemplo) ou carregue de outra fonte:
    # df['min_tensao (kPa)'] = [315, 520, 408, 530, 330]
    # df['max_tensao (kPa)'] = [710, 1305, 3000, 1255, 680]
    raise KeyError(
        "Colunas de esforços 'Fz', 'Mx' e 'My' não encontradas no Excel. "
        "Inclua-as ou preencha manualmente 'min_tensao (kPa)' e 'max_tensao (kPa)'."
    )

# 4) Conferir colunas necessárias
cols = ["min_tensao (kPa)", "max_tensao (kPa)", "sigma_adm (kPa)"]
faltando = [c for c in cols if c not in df.columns]
if faltando:
    raise KeyError(f"Colunas ausentes: {faltando}")

# 5) Aplicar as fórmulas (sua regra)
df["g_0"] = np.where(
    df["min_tensao (kPa)"] < 0.0,
    np.abs(df["min_tensao (kPa)"] / df["sigma_adm (kPa)"]),
    1.15 * df["min_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
).astype(float)

df["g_1"] = np.where(
    df["max_tensao (kPa)"] < 0.0,
    np.abs(df["max_tensao (kPa)"] / df["sigma_adm (kPa)"]),
    1.15 * df["max_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
).astype(float)

# 6) Ver e salvar
print(df[["h_x (m)", "h_y (m)", "Fz", "Mx", "My",
          "min_tensao (kPa)", "max_tensao (kPa)", "sigma_adm (kPa)", "g_0", "g_1"]].head())

df.to_excel(ARQ_OUT, index=False)
print(f"Arquivo '{ARQ_OUT}' salvo.")


KeyError: "Colunas de esforços 'Fz', 'Mx' e 'My' não encontradas no Excel. Inclua-as ou preencha manualmente 'min_tensao (kPa)' e 'max_tensao (kPa)'."

# verificando o processo de otimização

In [3]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

# === SUAS FUNÇÕES REAIS ===
from my_example import *

# === PYMOO ===
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize

# -*- coding: utf-8 -*-
"""
Otimização de fundações com pymoo (GA)
- Lê 'teste_sem_sobreposicao.xlsx'
- Importa funções reais de 'my_example.py'
- Define obj(x) SEM estado global
- Monta g_0..g_15 + penalização agregada df['g']
- Roda GA, e se não houver solução factível, devolve a melhor por CV (menor violação)
- EXTRA: imprime, por fundação, as g_k > 0 **para o melhor X** (somente ele)
- DISCRETO: h_x, h_y, h_z em passos de 0.1 (com clip nos limites)
- EXPORTA: melhor_X.xlsx com:
    * aba 'melhor_X'   -> h_x, h_y, h_z
    * aba 'restricoes' -> h_x, h_y, h_z, g_0..g_15, g_pos_sum, violadas
"""

# ==========================
# 0) DADOS BASE IMUTÁVEIS
# ==========================
N_COMB = 3  # c1, c2, c3
PENAL = 1e6
STEP = 0.1  # passo de discretização
XL = 1.0    # limite inferior (coerente com o problema)
XU = 5.0    # limite superior (coerente com o problema)

df_base = tensao_adm_solo(pd.read_excel("teste_reduzido.xlsx")).reset_index(drop=True)
N_FUNDACOES = len(df_base)

# Checagens mínimas
for k in range(1, N_COMB + 1):
    for name in (f"Fz-c{k}", f"Mx-c{k}", f"My-c{k}"):
        if name not in df_base.columns:
            raise RuntimeError(f"Coluna obrigatória ausente no Excel: '{name}'")
need_cols = ["xg (m)", "yg (m)", "ap (m)", "bp (m)", "sigma_adm (kPa)"]
miss = [c for c in need_cols if c not in df_base.columns]
if miss:
    raise RuntimeError("Colunas obrigatórias ausentes no Excel: " + ", ".join(miss))

# ==============================
# UTIL: SNAP DISCRETO (0.1)
# ==============================
def _snap_array(a, step=STEP, lo=XL, hi=XU):
    a = np.asarray(a, dtype=float)
    return np.clip(np.round(a / step) * step, lo, hi)

def _snap_xyz(x_full):
    """
    Recebe o vetor X completo (h_x ... h_y ... h_z ...) e devolve X quantizado (0.1) e 'clipado'.
    """
    x_full = np.asarray(x_full, dtype=float).copy()
    hx = _snap_array(x_full[0:N_FUNDACOES])
    hy = _snap_array(x_full[N_FUNDACOES:2*N_FUNDACOES])
    hz = _snap_array(x_full[2*N_FUNDACOES:3*N_FUNDACOES])
    return np.hstack([hx, hy, hz])

# ==============================
# 1) FUNÇÃO-OBJETIVO (SEM ESTADO)
# ==============================
def obj(x: np.ndarray):
    """
    Retorna:
        F (float): soma dos volumes
        G (np.ndarray [N_FUNDACOES,]): penalização agregada por fundação (>=0 quando viola)
    """
    # === DISCRETO ===
    xq = _snap_xyz(x)

    df = df_base.copy(deep=True)
    hx = np.asarray(xq[0:N_FUNDACOES], dtype=float)
    hy = np.asarray(xq[N_FUNDACOES:2 * N_FUNDACOES], dtype=float)
    hz = np.asarray(xq[2 * N_FUNDACOES:3 * N_FUNDACOES], dtype=float)

    df["h_x (m)"] = hx
    df["h_y (m)"] = hy
    df["h_z (m)"] = hz

    # Volume
    df["volume (m3)"] = df["h_x (m)"] * df["h_y (m)"] * df["h_z (m)"]

    # Tensões críticas
    max_tensoes, min_tensoes = [], []
    rotulos_comb = [f"c{i}" for i in range(1, N_COMB + 1)]
    idx_crit_max, idx_crit_min = [], []
    fz_max = []; mx_max = []; my_max = []
    fz_min = []; mx_min = []; my_min = []

    for _, row in df.iterrows():
        tmax_aux, tmin_aux = [], []
        for i in range(1, N_COMB + 1):
            smax, smin = calcular_sigma_max(
                row[f"Fz-c{i}"], row[f"Mx-c{i}"], row[f"My-c{i}"],
                row["h_x (m)"], row["h_y (m)"]
            )
            tmax_aux.append(smax); tmin_aux.append(smin)

        mxv = max(tmax_aux)
        mnv = min(tmin_aux)
        max_tensoes.append(mxv)
        min_tensoes.append(mnv)
        imax = int(np.argmax(tmax_aux))
        imin = int(np.argmin(tmin_aux))
        idx_crit_max.append(rotulos_comb[imax]); idx_crit_min.append(rotulos_comb[imin])
        comb_max = rotulos_comb[imax]; comb_min = rotulos_comb[imin]
        fz_max.append(row[f"Fz-{comb_max}"]); mx_max.append(row[f"Mx-{comb_max}"]); my_max.append(row[f"My-{comb_max}"])
        fz_min.append(row[f"Fz-{comb_min}"]); mx_min.append(row[f"Mx-{comb_min}"]); my_min.append(row[f"My-{comb_min}"])

    df["max_tensao (kPa)"] = np.asarray(max_tensoes, dtype=float)
    df["min_tensao (kPa)"] = np.asarray(min_tensoes, dtype=float)

    df["comb_max_tensao"] = idx_crit_max
    df["Fz_max (kN)"] = np.asarray(fz_max, dtype=float)
    df["Mx_max (kN.m)"] = np.asarray(mx_max, dtype=float)
    df["My_max (kN.m)"] = np.asarray(my_max, dtype=float)
    df["comb_min_tensao"] = idx_crit_min
    df["Fz_min (kN)"] = np.asarray(fz_min, dtype=float)
    df["Mx_min (kN.m)"] = np.asarray(mx_min, dtype=float)
    df["My_min (kN.m)"] = np.asarray(my_min, dtype=float)

    # Restrições geom. (balanço)
    g0_geo_ps, g1_geo_ps, g2_geo_ps, g3_geo_ps = [], [], [], []
    cap_list, cbp_list = [], []
    for _, row in df.iterrows():
        cap, cbp, g0, g1, g2, g3 = restricao_geometrica_balanco_pilar_sapata(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"], h_z=row["h_z (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"]
        )
        cap_list.append(cap); cbp_list.append(cbp)
        g0_geo_ps.append(g0); g1_geo_ps.append(g1)
        g2_geo_ps.append(g2); g3_geo_ps.append(g3)

    df["cap (m)"] = np.asarray(cap_list, dtype=float)
    df["cbp (m)"] = np.asarray(cbp_list, dtype=float)

    df["g_2"] = np.asarray(g0_geo_ps, dtype=float)
    df["g_3"] = np.asarray(g1_geo_ps, dtype=float)
    df["g_4"] = np.asarray(g2_geo_ps, dtype=float)
    df["g_5"] = np.asarray(g3_geo_ps, dtype=float)

    # Restrições geom. pilar-sapata
    g0_geo_p, g1_geo_p = [], []
    for _, row in df.iterrows():
        g0, g1 = restricao_geometrica_pilar_sapata(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"]
        )
        g0_geo_p.append(g0); g1_geo_p.append(g1)
    df["g_6"] = np.asarray(g0_geo_p, dtype=float)
    df["g_7"] = np.asarray(g1_geo_p, dtype=float)

    # Punção – usa comb. crítica de máximo
    ro = 0.01; cob = 0.025; fck = 25000.0; fcd = fck / 1.4
    gp0_list= []; gp1_list= []; gp2_list= []; gp3_list= []
    gp4_list= []; gp5_list= []; gp6_list= []
    for _, row in df.iterrows():
        ke, kx, ky, wpx, wpy, u, talsd1, talrd1, talsd2, talrd2, g0, g1, g2, g3, g4, g5, g6 = restricao_puncao(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"], h_z=row["h_z (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"] ,
            f_z=row["Fz_max (kN)"], m_x=row["Mx_max (kN.m)"], m_y=row["My_max (kN.m)"],
            ro=ro, cob=cob, fck=fck, fcd=fcd
        )
        gp0_list.append(g0); gp1_list.append(g1); gp2_list.append(g2)
        gp3_list.append(g3); gp4_list.append(g4); gp5_list.append(g5); gp6_list.append(g6)

    df["g_8"]  = np.asarray(gp0_list, dtype=float)
    df["g_9"]  = np.asarray(gp1_list, dtype=float)
    df["g_10"] = np.asarray(gp2_list, dtype=float)
    df["g_11"] = np.asarray(gp3_list, dtype=float)
    df["g_12"] = np.asarray(gp4_list, dtype=float)
    df["g_13"] = np.asarray(gp5_list, dtype=float)
    df["g_14"] = np.asarray(gp6_list, dtype=float)

    # Interseção
    xa = df["xg (m)"] + df["h_x (m)"] / 2.0
    ya = df["yg (m)"] + df["h_y (m)"] / 2.0
    xb = df["xg (m)"] - df["h_x (m)"] / 2.0
    yb = df["yg (m)"] - df["h_y (m)"] / 2.0
    xc = df["xg (m)"] - df["h_x (m)"] / 2.0
    yc = df["yg (m)"] + df["h_y (m)"] / 2.0
    xd = df["xg (m)"] + df["h_x (m)"] / 2.0
    yd = df["yg (m)"] - df["h_y (m)"] / 2.0

    polys = []
    for i in range(len(df)):
        pts = [(xa.iloc[i], ya.iloc[i]), (xb.iloc[i], yb.iloc[i]),
               (xc.iloc[i], yc.iloc[i]), (xd.iloc[i], yd.iloc[i]), (xa.iloc[i], ya.iloc[i])]
        polys.append(Polygon(pts))

    inter_count = np.zeros(len(df), dtype=float)
    for i in range(len(df)):
        for j in range(len(df)):
            if i != j and polys[i].intersects(polys[j]):
                inter_count[i] += 1.0
    df["g_15"] = inter_count

    # Tensões -> g_0 e g_1 (sua fórmula)
    df["g_0"] = np.where(
        df["min_tensao (kPa)"] < 0.0,
        np.abs(df["min_tensao (kPa)"] / df["sigma_adm (kPa)"]),
        1.15 * df["min_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
    ).astype(float)
    df["g_1"] = np.where(
        df["max_tensao (kPa)"] < 0.0,
        np.abs(df["max_tensao (kPa)"] / df["sigma_adm (kPa)"]),
        1.15 * df["max_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
    ).astype(float)

    # Garantir todas g_0..g_15
    req_cols = [f"g_{k}" for k in range(16)]
    missing = [c for c in req_cols if c not in df.columns]
    if missing:
        raise RuntimeError("Colunas de restrição não geradas: " + ", ".join(missing))

    # Penalização agregada por fundação
    g_mat = df[req_cols].to_numpy(dtype=float)
    g_pos = np.where(g_mat > 0.0, g_mat, 0.0)
    df["g"] = PENAL * np.sum(g_pos, axis=1)

    F = float(np.sum(df["volume (m3)"].to_numpy(dtype=float)))
    G = df["g"].to_numpy(dtype=float).reshape(-1)

    if not np.isfinite(F) or np.any(~np.isfinite(G)):
        raise RuntimeError("F ou G contém NaN/Inf. Verifique cálculos intermediários.")

    return F, G

# ==============================
# 1.1) RELATÓRIO DE g_k (MELHOR X, DISCRETO)
# ==============================
def _constraints_df_from_x(x: np.ndarray) -> pd.DataFrame:
    """
    Constrói um DataFrame com as colunas g_0..g_15 para o vetor x fornecido.
    (Sem a agregação PENAL; serve somente para relatório das violações.)
    """
    xq = _snap_xyz(x)

    df = df_base.copy(deep=True)
    hx = np.asarray(xq[0:N_FUNDACOES], dtype=float)
    hy = np.asarray(xq[N_FUNDACOES:2 * N_FUNDACOES], dtype=float)
    hz = np.asarray(xq[2 * N_FUNDACOES:3 * N_FUNDACOES], dtype=float)

    df["h_x (m)"] = hx
    df["h_y (m)"] = hy
    df["h_z (m)"] = hz

    # Tensões críticas (necessárias para punção)
    max_tensoes, min_tensoes = [], []
    rotulos_comb = [f"c{i}" for i in range(1, N_COMB + 1)]
    fz_max = []; mx_max = []; my_max = []
    for _, row in df.iterrows():
        tmax_aux, tmin_aux = [], []
        for i in range(1, N_COMB + 1):
            smax, smin = calcular_sigma_max(
                row[f"Fz-c{i}"], row[f"Mx-c{i}"], row[f"My-c{i}"],
                row["h_x (m)"], row["h_y (m)"]
            )
            tmax_aux.append(smax); tmin_aux.append(smin)
        mxv = max(tmax_aux); mnv = min(tmin_aux)
        max_tensoes.append(mxv); min_tensoes.append(mnv)
        imax = int(np.argmax(tmax_aux))
        comb_max = rotulos_comb[imax]
        fz_max.append(row[f"Fz-{comb_max}"]); mx_max.append(row[f"Mx-{comb_max}"]); my_max.append(row[f"My-{comb_max}"])

    df["max_tensao (kPa)"] = np.asarray(max_tensoes, dtype=float)
    df["min_tensao (kPa)"] = np.asarray(min_tensoes, dtype=float)
    df["Fz_max (kN)"] = np.asarray(fz_max, dtype=float)
    df["Mx_max (kN.m)"] = np.asarray(mx_max, dtype=float)
    df["My_max (kN.m)"] = np.asarray(my_max, dtype=float)

    # Geometria (balanço)
    g0_geo_ps, g1_geo_ps, g2_geo_ps, g3_geo_ps = [], [], [], []
    for _, row in df.iterrows():
        _, _, g0, g1, g2, g3 = restricao_geometrica_balanco_pilar_sapata(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"], h_z=row["h_z (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"]
        )
        g0_geo_ps.append(g0); g1_geo_ps.append(g1); g2_geo_ps.append(g2); g3_geo_ps.append(g3)
    df["g_2"] = np.asarray(g0_geo_ps, dtype=float)
    df["g_3"] = np.asarray(g1_geo_ps, dtype=float)
    df["g_4"] = np.asarray(g2_geo_ps, dtype=float)
    df["g_5"] = np.asarray(g3_geo_ps, dtype=float)

    # Geometria pilar-sapata
    g0_geo_p, g1_geo_p = [], []
    for _, row in df.iterrows():
        g0, g1 = restricao_geometrica_pilar_sapata(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"]
        )
        g0_geo_p.append(g0); g1_geo_p.append(g1)
    df["g_6"] = np.asarray(g0_geo_p, dtype=float)
    df["g_7"] = np.asarray(g1_geo_p, dtype=float)

    # Punção (usa comb. crítica de máximo)
    ro = 0.01; cob = 0.025; fck = 25000.0; fcd = fck / 1.4
    gp0_list= []; gp1_list= []; gp2_list= []; gp3_list= []
    gp4_list= []; gp5_list= []; gp6_list= []
    for _, row in df.iterrows():
        _, _, _, _, _, _, _, _, _, _, g0, g1, g2, g3, g4, g5, g6 = restricao_puncao(
            h_x=row["h_x (m)"], h_y=row["h_y (m)"], h_z=row["h_z (m)"],
            a_p=row["ap (m)"], b_p=row["bp (m)"],
            f_z=row["Fz_max (kN)"], m_x=row["Mx_max (kN.m)"], m_y=row["My_max (kN.m)"],
            ro=ro, cob=cob, fck=fck, fcd=fcd
        )
        gp0_list.append(g0); gp1_list.append(g1); gp2_list.append(g2)
        gp3_list.append(g3); gp4_list.append(g4); gp5_list.append(g5); gp6_list.append(g6)
    df["g_8"]  = np.asarray(gp0_list, dtype=float)
    df["g_9"]  = np.asarray(gp1_list, dtype=float)
    df["g_10"] = np.asarray(gp2_list, dtype=float)
    df["g_11"] = np.asarray(gp3_list, dtype=float)
    df["g_12"] = np.asarray(gp4_list, dtype=float)
    df["g_13"] = np.asarray(gp5_list, dtype=float)
    df["g_14"] = np.asarray(gp6_list, dtype=float)

    # Interseção de sapatas
    xa = df["xg (m)"] + df["h_x (m)"] / 2.0
    ya = df["yg (m)"] + df["h_y (m)"] / 2.0
    xb = df["xg (m)"] - df["h_x (m)"] / 2.0
    yb = df["yg (m)"] - df["h_y (m)"] / 2.0
    xc = df["xg (m)"] - df["h_x (m)"] / 2.0
    yc = df["yg (m)"] + df["h_y (m)"] / 2.0
    xd = df["xg (m)"] + df["h_x (m)"] / 2.0
    yd = df["yg (m)"] - df["h_y (m)"] / 2.0

    polys = []
    for i in range(len(df)):
        pts = [(xa.iloc[i], ya.iloc[i]), (xb.iloc[i], yb.iloc[i]),
               (xc.iloc[i], yc.iloc[i]), (xd.iloc[i], yd.iloc[i]), (xa.iloc[i], ya.iloc[i])]
        polys.append(Polygon(pts))

    inter_count = np.zeros(len(df), dtype=float)
    for i in range(len(df)):
        for j in range(len(df)):
            if i != j and polys[i].intersects(polys[j]):
                inter_count[i] += 1.0
    df["g_15"] = inter_count

    # Tensões -> g0/g1
    df["g_0"] = np.where(
        df["min_tensao (kPa)"] < 0.0,
        np.abs(df["min_tensao (kPa)"] / df["sigma_adm (kPa)"]),
        1.15 * df["min_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
    ).astype(float)
    df["g_1"] = np.where(
        df["max_tensao (kPa)"] < 0.0,
        np.abs(df["max_tensao (kPa)"] / df["sigma_adm (kPa)"]),
        1.15 * df["max_tensao (kPa)"] / df["sigma_adm (kPa)"] - 1.0
    ).astype(float)

    return df

def _print_violated_g_for_best_x(x: np.ndarray):
    """
    Imprime SOMENTE as restrições g_k > 0 calculadas para o MELHOR X (vetor fornecido).
    """
    df_det = _constraints_df_from_x(x)
    req_cols = [f"g_{k}" for k in range(16)]

    print("\nRestrições violadas (g_k > 0) por fundação — para o melhor X:")
    any_violation = False
    for i, row in df_det[req_cols].iterrows():
        viols = [(col, float(row[col])) for col in req_cols if float(row[col]) > 0.0]
        if viols:
            any_violation = True
            parts = [f"{name}={val:.6g}" for name, val in viols]
            print(f"  Fundação #{i}: " + ", ".join(parts))
    if not any_violation:
        print("  Nenhuma violação (todas g_k ≤ 0).")

# ==============================
# 2) PROBLEMA PYMOO
# ==============================
class MyProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=N_FUNDACOES * 3,
            n_obj=1,
            n_constr=N_FUNDACOES,
            xl=np.array([XL] * N_FUNDACOES * 3, dtype=float),
            xu=np.array([XU] * N_FUNDACOES * 3, dtype=float),
        )

    def _evaluate(self, x, out, *args, **kwargs):
        f, g = obj(x)
        out["F"] = float(f)
        out["G"] = np.asarray(g, dtype=float).reshape(-1)

# ==============================
# 3) SEEDING AMIGÁVEL + GA
# ==============================
def build_seed():
    hx0 = np.maximum(1.0, df_base["ap (m)"].to_numpy(dtype=float) + 0.20)
    hy0 = np.maximum(1.0, df_base["bp (m)"].to_numpy(dtype=float) + 0.20)
    hz0 = np.full(N_FUNDACOES, 1.0, dtype=float)
    x0 = np.hstack([hx0, hy0, hz0])
    x0 = _snap_xyz(np.clip(x0, XL, XU))
    return x0

def inject_seed(pop_array, seed):
    pop_array[0, :] = seed
    return pop_array

# ==============================
# 4) EXECUÇÃO
# ==============================
if __name__ == "__main__":
    problem = MyProblem()
    algorithm = GA(
        pop_size=40,
        n_offsprings=20,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PM(eta=20),
        eliminate_duplicates=True,
    )

    res = minimize(
        problem,
        algorithm,
        ("n_gen", 40),
        seed=1,
        verbose=False,
        save_history=True,
    )

    # ==============================
    # 5) EXTRAÇÃO ROBUSTA DO “MELHOR”
    # ==============================
    def get_best_solution(res):
        if res.F is not None and res.X is not None:
            return res.X, res.F, res.G, True

        pop = getattr(res, "pop", None)
        if pop is None and hasattr(res, "history") and len(res.history) > 0:
            pop = res.history[-1].pop
        if pop is None:
            return None, None, None, False

        CV = pop.get("CV").ravel()
        I = int(np.argmin(CV))
        Xb = pop[I].get("X")
        Fb = pop[I].get("F")
        Gb = pop[I].get("G")
        return Xb, Fb, Gb, False

    X, F, G, feasible = get_best_solution(res)
    Xq = _snap_xyz(X) if X is not None else None

    print("\n================ RESULTADOS ================")
    print("Encontrou solução factível?", feasible)
    if Xq is not None:
        print("Melhor F (volume total):", F)
        if G is not None:
            print("Max G:", np.max(G))
            print("Todas restrições satisfeitas? (G <= 0):", np.all(G <= 0))
        print("Vetor X (h_x ... h_y ... h_z ...):\n", Xq)
    else:
        print("Não foi possível recuperar solução (nem por CV). Revise restrições/dados.")

    # === Relatório adicional: SOMENTE o correspondente ao MELHOR X (discreto) ===
    if Xq is not None:
        _print_violated_g_for_best_x(Xq)

        # ======= NOVO: exportar melhor_X + restrições para Excel =======
        try:
            hx = np.asarray(Xq[0:N_FUNDACOES], dtype=float)
            hy = np.asarray(Xq[N_FUNDACOES:2*N_FUNDACOES], dtype=float)
            hz = np.asarray(Xq[2*N_FUNDACOES:3*N_FUNDACOES], dtype=float)

            # Aba 1: melhor_X
            df_out = pd.DataFrame({
                "h_x (m)": hx,
                "h_y (m)": hy,
                "h_z (m)": hz
            })

            # Aba 2: restricoes (g_0..g_15 + resumo)
            df_g = _constraints_df_from_x(Xq).copy()
            req_cols = [f"g_{k}" for k in range(16)]
            # garante ordem e inclui h_x,h_y,h_z no início
            for c in ["h_x (m)", "h_y (m)", "h_z (m)"]:
                if c not in df_g.columns:
                    df_g[c] = np.nan
            df_g = df_g[["h_x (m)", "h_y (m)", "h_z (m)"] + req_cols]

            # soma de violações positivas e lista das violadas
            g_mat = df_g[req_cols].to_numpy(dtype=float)
            g_pos_sum = np.sum(np.where(g_mat > 0.0, g_mat, 0.0), axis=1)
            violadas = []
            for i, row in df_g[req_cols].iterrows():
                v = [col for col in req_cols if float(row[col]) > 0.0]
                violadas.append("; ".join(v) if v else "")

            df_g["g_pos_sum"] = g_pos_sum
            df_g["violadas"] = violadas

            # grava em Excel (openpyxl -> xlsxwriter fallback)
            try:
                with pd.ExcelWriter("melhor_X.xlsx", engine="openpyxl") as writer:
                    df_out.to_excel(writer, index=False, sheet_name="melhor_X")
                    df_g.to_excel(writer, index=False, sheet_name="restricoes")
            except Exception:
                with pd.ExcelWriter("melhor_X.xlsx", engine="xlsxwriter") as writer:
                    df_out.to_excel(writer, index=False, sheet_name="melhor_X")
                    df_g.to_excel(writer, index=False, sheet_name="restricoes")

            print("Arquivo 'melhor_X.xlsx' salvo (abas: 'melhor_X' e 'restricoes').")
        except Exception as e:
            print("Não foi possível salvar 'melhor_X.xlsx':", e)



================ RESULTADOS ================
Encontrou solução factível? False
Melhor F (volume total): [636.815]
Max G: 7863455.442583732
Todas restrições satisfeitas? (G <= 0): False
Vetor X (h_x ... h_y ... h_z ...):
 [2.2 4.  2.  3.1 1.7 3.8 3.4 2.8 3.8 1.9 1.3 2.5 1.8 2.2 3.  3.3 2.1 2.4
 3.2 3.3 2.8 2.  4.  4.6 3.2 3.  2.4 4.6 2.8 3.5 1.9 1.8 3.5 4.8 2.5 1.2
 4.8 3.8 3.4 4.  4.6 2.8 3.2 2.  2.6 4.9 2.5 4.9 1.7 3.4 2.  2.6 4.8 2.5
 2.4 3.  1.2 2.6 3.8 3.4 4.5 1.5 3.4 3.4 3.3 1.8 1.8 2.7 2.6 3.3 3.7 4.2
 2.3 4.1 2.4 1.3 2.2 1.6 2.4 3.1 1.8 1.8 1.8 1.2 2.9 1.1 3.2 1.1 3.  1.9
 2.  1.6 1.1 3.3 1.3 2.1 1.5 1.4 1.8 1.2 2.1 1.4 1.5 1.2 1.1 1.2 3.  2.2
 1.5 4.8 1.9]

Restrições violadas (g_k > 0) por fundação — para o melhor X:
  Fundação #0: g_4=0.230769, g_8=0.157813, g_9=0.54375
  Fundação #1: g_8=0.179375, g_15=1
  Fundação #2: g_4=0.257143, g_9=0.5225, g_15=1
  Fundação #3: g_8=0.122188, g_9=0.122188
  Fundação #4: g_4=0.655172, g_5=1.28571
  Fundação #5: g_4=0.823529, g_5=0.050847